# Prepping COCA
In this notebook, I'll prepare relevant COCA sections to train the Roark Parser.
## Table of Contents
1. [Loading COCA](#1)
2. [Test Run](#2)
3. [Full Run](#3)
<a id='1'>

## Loading COCA

In [1]:
def openTextFile(filename):
    text = []
    file = open(filename)
    lines = file.read() #char by char
    file.close()
    return lines

In [2]:
import os
root = 'D:/COCA/stim/text_newspaper_lsp/'
text_files = os.listdir(root)
sample = openTextFile(root+text_files[0]) #one file is 25 million characters
print(len(sample))
mini = sample[:25000] #this is 1/1000th of one file. there are about 25 files in this directory, and we were talking about using 4 directories. so this is about 1/100,000th of the total text we can use.
#mini

24273875


<a id='2'>

## Test Run

In [3]:
import nltk
import pandas as pd
from stanfordcorenlp import StanfordCoreNLP
from nltk import StanfordPOSTagger
from nltk.parse import stanford
from nltk.parse import CoreNLPParser
parser = CoreNLPParser(url='http://localhost:9000')

In [4]:
def cleanText(text):
    sents = nltk.sent_tokenize(text)
    clean_sents = []
    for sent in sents:
        if '@' in sent:
            continue
        if '<p>' in sent:
            sent = sent.split('<p>')[1]
        #take out low k-band words
        clean_sents.append(sent)
    return clean_sents

In [5]:
def growTrees(clean_sents):

    forest = []
    for sent in clean_sents:
        trees = parser.raw_parse(sent)
        for tree in trees:
            forest.append(tree)    
    return forest

In [8]:
%time sents = cleanText(mini) #1
%time parses = growTrees(sents) #10000
print(len(sents))
print(len(parses))
sents[:10]

Wall time: 15 ms
Wall time: 34.6 s
174
174


[' Some teams improve by adding players .',
 'Others make themselves better by getting rid of players .',
 'Addition by subtraction , Branch Rickey called it .',
 'Then there are the Yankees , who hope to prove that a team that won more games than any other in baseball history can at least repeat as champions , if not match or exceed that victory total , without making any changes .',
 " When the Yankees ' pitchers and catchers have their first spring training workout in Tampa , Fla. , on Thursday and when the rest of the World Series winners join them Feb. 23 , they will not have to introduce themselves to each other .",
 'They are the same guys who dressed and played together last season .',
 ' True , Tim Raines will not be there , but he is the lone exception .',
 'Every other member of the post-season team will be back in all the old familiar places .',
 'Bernie Williams , who seemed certain to leave , stayed when the numbers after the dollar sign got big enough .',
 " General Mana

In [16]:
import pickle
file = open('mini_train.pkl', 'wb')
pickle.dump(parses, file)
file.close()

<a id='3'>

## Full Run